In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Specification\n",
    "\n",
    "$$\n",
    "\\def\\vec#1{\\mathbf{#1}}\n",
    "\\def\\T{\\mathsf{T}}\n",
    "\\def\\F{\\mathsf{F}}\n",
    "\\def\\U{\\mathsf{U}}\n",
    "\\def\\popcount{\\mathtt{popcount}}\n",
    "\\def\\count{\\mathtt{count}}\n",
    "\\def\\fhd{\\mathtt{fhd}}\n",
    "\\def\\vsum{\\mathtt{sum}}\n",
    "$$\n",
    "\n",
    "First we need some definitions and theory on masked bitvectors and their representation in $ℤ_{2^{16}}$ and present a simple secure multiparty computation scheme.\n",
    "\n",
    "## Masked bitvectors\n",
    "\n",
    "### Binary operations in rings\n",
    "\n",
    "Given a bitvector $\\vec b ∈ \\{\\F,\\T\\}^n$ of length $n$, we take the usual binary operations of *not* $¬$, *and* $∧$, *or* $∨$, *xor* $⊕$, and also $\\popcount$. We can embed this in a ring $R$ by representing $\\F,\\T$ as $0,1$ respectively and using the following operations on vectors $\\vec b ∈ R^n$:\n",
    "\n",
    "$$\n",
    "\\begin{aligned}\n",
    "¬ \\vec a &= 1 - \\vec a \\\\\n",
    "\\vec a ∧ \\vec b &= \\vec a ⊙ \\vec b \\\\\n",
    "\\vec a ∨ \\vec b &= \\vec a + \\vec b - \\vec a ⊙ \\vec b \\\\\n",
    "\\vec a ⊕ \\vec b &= \\vec a + \\vec b - 2 ⋅ \\vec a ⊙ \\vec b \\\\\n",
    "\\popcount(\\vec a) &= \\vsum(\\vec a) \\\\\n",
    "\\end{aligned}\n",
    "$$\n",
    "\n",
    "Where $⊙$ is the element wise product and $\\vsum$ adds the vector elements. Note that $\\vsum(\\vec a ⊙ \\vec b)$ is the vector dot product $\\vec a ⋅ \\vec b$. Note also that the $\\popcount$ will be computed in the ring, so a sufficiently large modular ring is required.\n",
    "\n",
    "### Masked binary operations in rings\n",
    "\n",
    "A *masked bitvector* has three values $\\mathtt{b} ∈ \\{\\F, \\T, \\U\\}^n$ which should be interpreted as *false*, *true* and *unavailable* respectively. The operations are the same as for binary, except when one of the arguments is $\\U$, the result is always $\\U$. We take $\\popcount$ to count the number of $\\T$'s and introduce $\\mathtt{count}$ to count the number of available entries i.e. either $\\F$ or $\\T$ but not $\\U$.\n",
    "\n",
    "We can represent this on a suitably large (**Q** how large?) ring as $-1,0,1$ for $\\F, \\U, \\T$ respectively.\n",
    "\n",
    "$$\n",
    "\\begin{aligned}\n",
    "¬ \\vec a &= - \\vec a \\\\\n",
    "\\vec a ∧ \\vec b &= ½ ⋅ \\vec a ⊙ \\vec b ⊙ (1 + \\vec a + \\vec b - \\vec a ⊙ \\vec b) \\\\\n",
    "\\vec a ∨ \\vec b &= ½ ⋅ \\vec a ⊙ \\vec b ⊙ (\\vec a ⊙ \\vec b + \\vec a + \\vec b -1 ) \\\\\n",
    "\\vec a ⊕ \\vec b &= -\\vec a ⊙ \\vec b \\\\\n",
    "\\count(\\vec a) &= \\vsum\\left(\\vec a^{⊙2}\\right) \\\\\n",
    "\\popcount(\\vec a) &= ½ ⋅ \\vsum\\left(\\vec a^{⊙2} + \\vec a\\right)\n",
    "\\end{aligned}\n",
    "$$\n",
    "\n",
    "Note that squaring, $\\vec a^{⊙2}$, produces a regular $0,1$ bitvector that is $0$ whenever the value is $\\U$ and $1$ otherwise, i.e. it  allows us to extract the mask as a regular bitvector. Similarly $½ ⋅(\\vec a^{⊙2} + \\vec a)$ extracts the data bits: $1$ for $\\T$ and $0$ otherwise. The reverse mapping, converting data bits $\\vec b$ and mask bits $\\vec m$ to a masked bitvector is $\\vec m - 2⋅\\vec b⊙\\vec m$. If the data bits are known to be $\\F$ in the in the unavailable region, then it simplifies to $\\vec m - 2⋅\\vec b⊙\\vec m$.\n",
    "\n",
    "In this representation *and* and *or* have awkward fourth degree expressions, though they can be evaluated using only two multiplies. The expressions for *xor*, $\\count$ and $\\popcount$ are quite nice considering that they correctly account for masks. This makes it a suitable system for computing fractional hamming distances.\n",
    "\n",
    "### Fractional hamming distance\n",
    "\n",
    "The *fractional hamming weight* of a masked bitvector $\\vec a$ is defined as\n",
    "\n",
    "$$\n",
    "\\begin{aligned}\n",
    "\\mathtt{fhw}(\\vec a) &= \\frac\n",
    "{\\popcount(\\vec a)}\n",
    "{\\count(\\vec a )}\n",
    "\\end{aligned}\n",
    "$$\n",
    "\n",
    "and the *fractional hamming distance* between two masked bitvectors $\\vec a$ and $\\vec b$ as\n",
    "\n",
    "$$\n",
    "\\begin{aligned}\n",
    "\\fhd(\\vec a, \\vec b)\n",
    "&= \\mathtt{fhw}(\\vec a ⊕ \\vec b) =\\frac\n",
    "{\\popcount(\\vec a ⊕ \\vec b)}\n",
    "{\\count(\\vec a ⊕ \\vec b)}\n",
    "\\end{aligned}\n",
    "$$\n",
    "\n",
    "In the ring representation these can be computed as\n",
    "\n",
    "$$\n",
    "\\begin{aligned}\n",
    "\\fhd(\\vec a, \\vec b)\n",
    "& =\\frac\n",
    "{ ½ ⋅ \\vsum\\left((-\\vec a ⊙ \\vec b)^{⊙2} -\\vec a ⊙ \\vec b\\right)}\n",
    "{\\vsum\\left((-\\vec a ⊙ \\vec b)^{⊙2}\\right)} \n",
    "&&=\\frac{1}{2} -\\frac\n",
    "{ \\vsum\\left(\\vec a ⊙ \\vec b\\right)}\n",
    "{2⋅\\vsum\\left((\\vec a ⊙ \\vec b)^{⊙2}\\right)}  \\\\\n",
    "\\end{aligned}\n",
    "$$\n",
    "\n",
    "Note that $(\\vec a ⊙ \\vec b)^{⊙2} = \\vec a^{⊙2} ⊙ \\vec b^{⊙2}$ and thus depends only on the masks of $\\vec a$ and $\\vec b$. Given the masks $\\vec a_m$ and $\\vec b_m$ it can be computed in binary as\n",
    "\n",
    "$$\n",
    "\\vsum\\left((\\vec a ⊙ \\vec b)^{⊙2}\\right) = \\popcount(\\vec a_m ∧ \\vec b_m)\n",
    "$$\n",
    "\n",
    "## Secure Multiparty Computation\n",
    "\n",
    "We construct a simple secret sharing scheme over $ℤ_{2^{16}}$. To encrypt a value $x$ we compute $n$ random secret shares $x_0,…,x_{n-1}$ such that\n",
    "\n",
    "$$\n",
    "x = \\sum_{i∈[0,n)} x_i\n",
    "$$\n",
    "\n",
    "One way to do this is by generating $n-1$ random values $x_0,…,x_{n-2}∈ℤ_{2^{16}}$ and solving for $x_{n-1}$:\n",
    "\n",
    "$$\n",
    "x_{n-1} = x - \\sum_{i∈[0,n-1)} x_i\n",
    "$$\n",
    "\n",
    "The shares are encrypted in the sense that this is essentially a [one-time-pad] with any $n-1$ shares being the decryption key for the remaining share. Another valid perspective is that the shares are independently random and the secret is stored in their correlation.\n",
    "\n",
    "[one-time-pad]: https://en.wikipedia.org/wiki/One-time_pad\n",
    "\n",
    "### Additive homomorphism\n",
    "\n",
    "This secret sharing scheme is additive homomorphic. Given two secrets $a$ and $b$ with shares $a_i$ and $b_i$, each party can locally compute $c_i = a_i + b_i$. These $c_i$ are valid shares for the sum $c = a + b$.\n",
    "\n",
    "Similarly we can compute the product of a secret $a$ with shares $a_i$ and a constant $b ∈ ℤ_{2^{16}}$ locally as $c_i = a_i ⋅ b$.\n",
    "\n",
    "Generalizing this, given a secret vector $\\vec a ∈ ℤ_{2^{16}}^n$ and a cleartext vector $\\vec b ∈ ℤ_{2^{16}}^n$ we can compute shares of the dot product $\\vec c = \\vec a ⋅ \\vec b$ as $c_i = \\sum_j a_{ij} \\cdot b_j$.\n",
    "\n",
    "Note that these newly created shares are correlated with the input secret shares. To solve this, they need to be re-randomized.\n",
    "\n",
    "### Re-randomizing\n",
    "\n",
    "To re-randomize results we construct secret shares of zero and add them to the result. One way to do this locally is for each party to have two random number generators, $r_0, …, r_{n-1}$ and some cyclic permutation $r_{σ(0)}, … r_{σ(n-1)}$. The a secret share $a_i$ is locally updated as:\n",
    "\n",
    "$$\n",
    "a_i' = a_i + r_i - r_{σ(i)}\n",
    "$$\n",
    "\n",
    "One method for parties $i$ and $σ(i)$ to establish shared randomness is by using [Diffie-Hellman][DH] to establish a seed for a cryptographic PRNG.\n",
    "\n",
    "[DH]: https://en.wikipedia.org/wiki/Diffie%E2%80%93Hellman_key_exchange\n",
    "\n",
    "## Iris codes\n",
    "\n",
    "For present purposes an iriscode is a $12\\ 800$-bit masked bitvector.\n",
    "\n",
    "### Rotations\n",
    "\n",
    "The $12\\ 800$ masked bitvectors can be interpreted as $64 × 200$ bit matrices. We can then define a rotation as a permutation on the columns:\n",
    "\n",
    "$$\n",
    "\\mathtt{rot}(\\vec b, n)[i,j] = \\vec b[i,(j+n)\\ \\mathrm{mod}\\ 200]\n",
    "$$\n",
    "\n",
    "### Distance\n",
    "\n",
    "The *distance* between two iriscodes $\\mathtt a$ and $\\mathtt b$ is defined as the minimum distance over rotations from $-15$ to $15$:\n",
    "\n",
    "$$\n",
    "\\mathtt{dist}(\\vec a, \\vec b) = \\min_{r∈[-15,15]}\\ \\fhd(\\mathtt{rot}(\\vec a, r), \\vec b)\n",
    "$$\n",
    "\n",
    "### Uniqueness\n",
    "\n",
    "To verify uniqueness we require that an iriscode $\\vec a$ is a certain minimum distance from all previous iriscodes:\n",
    "\n",
    "$$\n",
    "\\mathop{\\Large ∀}\\limits_{\\vec b ∈ \\mathtt{DB}}\\ \n",
    "\\mathtt{dist}(\\vec a, \\vec b) > \\mathrm{threshold}\n",
    "$$\n",
    "\n",
    "where $\\mathtt{DB}$ is the set of already registered iriscodes (currently 3 million entries).\n",
    "\n",
    "When there is a match, we are also interested in finding the location of the best match. Both can be addressed by implementing a method that returns the index of the minimum distance entry.\n",
    "\n",
    "## Iriscode SMPC v1\n",
    "\n",
    "Objective: DB iriscode bits encrypted. Query, mask and distances unencrypted. \n",
    "\n",
    "For each iriscode $\\vec b_i$ in the database $\\mathtt{DB}$ we convert it to a masked bitvector embedded in the ring $ℤ_{2^{16}}$. We then create secret shares such that each party $j$ has a share $\\vec b_{ij}$.\n",
    "\n",
    "Given a query iriscode $\\vec a$, also encoded as a masked bitvector embedded in $ℤ_{2^{16}}$, but not converted to secret shares. Each party $j$ computes rotations of the query $r∈[-15,15]$ and computes the following with each database share $\\vec b_{ij}$:\n",
    "\n",
    "$$\n",
    "d_{ijr} = \\vsum(\\mathtt{rot}(\\vec a, r) ⊙ \\vec b_{ij})\n",
    "$$\n",
    "\n",
    "Each party produces $31 ⋅ |\\mathtt{DB}|$ values this way. These values are aggregated in the coordinator, which can decrypt the results\n",
    "\n",
    "$$\n",
    "d_{ir} = \\sum_j d_{ijr} = \\vsum(\\mathtt{rot}(\\vec a, r) ⊙ \\vec b_{i})\n",
    "$$\n",
    "\n",
    "The coordinator has a database of only the masks $\\vec b_{i\\ \\mathrm{m}}$ as regular bitvectors. This allows it to compute\n",
    "\n",
    "$$\n",
    "m_{ir} = \\popcount(\\mathtt{rot}(\\vec a_{\\mathrm{m}}, r) ∧ \\vec b_{i\\,\\mathrm{m}})\n",
    "$$\n",
    "\n",
    "With both these pieces of information, the coordinator can compute the $\\fhd$ between the query $\\vec a$ and database entry $\\vec b_i$.\n",
    "\n",
    "$$\n",
    "\\mathtt{dist}(\\vec a, \\vec b_i) =\n",
    "\\min_{r∈[-15,15]}\\ \\left( \\frac12 - \\frac\n",
    "{d_{ir}}\n",
    "{2⋅m_{ir}} \\right)\n",
    "$$\n",
    "\n",
    "To see that this work we can substitute back all the definitions from above:\n",
    "\n",
    "$$\n",
    "\\begin{aligned}\n",
    "\\mathtt{dist}(\\vec a, \\vec b_i) &=\n",
    "\\min_{r∈[-15,15]}\\ \\left( \\frac12 - \\frac\n",
    "{d_{ir}}\n",
    "{2⋅m_{ir}} \\right) \\\\\n",
    "\\min_{r∈[-15,15]}\\ \\fhd(\\mathtt{rot}(\\vec a, r), \\vec b_i) &=\n",
    "\\min_{r∈[-15,15]}\\ \\left( \\frac12 - \\frac\n",
    "{\\sum_j d_{ijr}}\n",
    "{2⋅\\popcount(\\vec a_{\\mathrm{m}} ∧ \\vec b_{i\\,\\mathrm{m}})} \\right) \\\\\n",
    "\\fhd(\\mathtt{rot}(\\vec a, r), \\vec b_i) &=\n",
    "\\frac12 - \\frac\n",
    "{\\vsum(\\mathtt{rot}(\\vec a, r) ⊙ \\vec b_{i})}\n",
    "{2⋅\\vsum\\left((\\mathtt{rot}(\\vec a, r) ⊙ \\vec b_{i})^{⊙2}\\right)} \\\\\n",
    "\\end{aligned}\n",
    "$$\n",
    "\n",
    "## References\n",
    "\n",
    "* [MRZ15] Payman Mohassel, Mike Rosulek, and Ye Zhang (2015). Fast and Secure Three-party Computation: The Garbled Circuit Approach.\n",
    "* [AFL+16] Toshinori Araki, Jun Furukawa, Yehuda Lindell, Ariel Nof, and Kazuma Ohara (2016) High-Throughput Semi-Honest Secure Three-Party Computation with an Honest Majority.\n",
    "* [MZ17] Payman Mohassel and Yupeng Zhang (2017). SecureML: A System for Scalable Privacy-Preserving Machine Learning.\n",
    "* [MR18] Payman Mohassel and Peter Rindal (2018). ABY3: A Mixed Protocol Framework for Machine Learning.\n",
    "* [CRS19] Harsh Chaudhari, Rahul Rachuri, and Ajith Suresh (2019). Trident: Efficient 4PC Framework for Privacy Preserving Machine Learning.\n",
    "* [BCPS19] Megha Byali, Harsh Chaudhari, Arpita Patra, and Ajith Suresh (2019). FLASH: Fast and Robust Framework for Privacy-preserving Machine Learning.\n",
    "* [PS20] Arpita Patra and Ajith Suresh (2020). BLAZE: Blazing Fast Privacy-Preserving Machine Learning.\n",
    "* [PSSY20] Arpita Patra, Thomas Schneider, Ajith Suresh, and Hossein Yalame (2020). ABY2.0: Improved Mixed-Protocol Secure Two-Party Computation.\n",
    "\n",
    "[MRZ15]: https://eprint.iacr.org/2015/931.pdf\n",
    "[AFL+16]: https://eprint.iacr.org/2016/768.pdf\n",
    "[MZ17]: https://eprint.iacr.org/2017/396.pdf\n",
    "[MR18]: https://eprint.iacr.org/2018/403.pdf\n",
    "[CRS19]: https://eprint.iacr.org/2019/1315.pdf\n",
    "[BCPS19]: https://eprint.iacr.org/2019/1365.pdf\n",
    "[PS20]: https://eprint.iacr.org/2020/042.pdf\n",
    "[PSSY20]: https://eprint.iacr.org/2020/1225.pdf\n",
    "\n",
    "\n"
   ]
  }
 ],
 "metadata": {
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 2
}